In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import datasets
from tqdm.notebook import tqdm

import utils

In [2]:
train_ds, valid_ds, test_ds = utils.load_dataset()

Found cached dataset civil_comments (/home/johnny/.cache/huggingface/datasets/civil_comments/default/0.9.0/e7a3aacd2ab7d135fa958e7209d10b1fa03807d44c486e3c34897aa08ea8ffab)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
score_npy = 'scores/ngram.npy'
score_column = 'scores'

bins = 5
cut = 'quantile'
allocation = 'pilot'

In [4]:
labels = test_ds['toxicity']
scores = np.load(score_npy)
df = pd.DataFrame(data={'toxicity' : labels, 'toxic' : test_ds['label'], 'scores' : scores})

In [5]:
df.head(1)

,toxicity,toxic,scores
0,0.0,0,0.017794


In [6]:
df['toxic'].sum(), df['toxic'].mean()

(106438, 0.05897253769515213)

In [7]:
size = 1000

In [8]:
p = df['toxic'].mean()
random_sampling_var = p * (1 - p)
np.sqrt(random_sampling_var / size)

0.007449481692906291

In [9]:
def get_error(df):
    sizes_sigmas = []
    for i, group in df.groupby('bin'):
        # to round up to 1
        pilot = group['toxic']
        sizes_sigmas.append((len(group), np.std(pilot)))

    allocations = []
    denominator = np.sum([ n_h * sigma_h for n_h, sigma_h in sizes_sigmas ])
    for n_h, sigma_h in sizes_sigmas:
        n_from_bin = size * n_h * sigma_h / denominator
        allocations.append(n_from_bin)
    
    stratified_var = 0
    for (i, group), n_from_bin in zip(df.groupby('bin'), allocations):

        p = group['toxic'].mean()

        # approximation when the groups are very large
        stratified_var += np.square(len(group) / len(df)) * (group['toxic'].var() / n_from_bin)
    return np.sqrt(stratified_var)

### Cutting

In [10]:
bins = 4
score_column = 'scores'

minimum, maximum = df[score_column].min(), df[score_column].max()

# equal width
# df['bin'] = pd.cut(df[score_column], np.linspace(minimum, maximum, num=bins+1), include_lowest=True)

# quantile binning
df['bin'] = pd.qcut(df[score_column], np.linspace(0, 1, num=bins+1))
get_error(df)

0.005191869649127323

In [11]:
minimum_bins = [0, 1]
minimum_err = 1
steps = 10

# for 4
indexes = [1, 1, 3]

for idx in indexes:
    bins = minimum_bins
    bins.insert(idx, 0)
    lb, ub = bins[idx-1], bins[idx+1]
    
    print(lb + (ub-lb)/steps, ub)
    
    for i in np.linspace( lb + (ub-lb)/steps, ub, steps, endpoint=False):
        bins[idx] = i
        df['bin'] = pd.qcut(df[score_column], bins)
        new_err = get_error(df)
        if new_err < minimum_err:
            minimum_err = new_err
            minimum_bins = bins
            
    print(bins)       
    print(new_err)
    print()

0.1 1
[0, 0.9099999999999999, 1]
0.005314325225981284

0.091 0.9099999999999999
[0, 0.8281, 0.9099999999999999, 1]
0.004928959762042162

0.9189999999999999 1
[0, 0.8281, 0.9099999999999999, 0.9919, 1]
0.004836806652555329



In [12]:
minimum_bins, minimum_err

([0, 0.8281, 0.9099999999999999, 0.9919, 1], 0.004776722309714379)

In [13]:
p = 0.05897253769515213
print(p, end=' ')

for within in [0.2, 0.1, 0.05]:
    desired_ci = p * within

    alpha = 0.05
    z_statistic = stats.norm.ppf(1 - (alpha / 2))
    desired_var = np.square(desired_ci / z_statistic)

    minimum = numerator / desired_var
    n = np.sum([minimum * multip for multip in multipliers])
    print(int(n+1), end=' ')

0.05897253769515213 

NameError: name 'numerator' is not defined